# Decoder-Only Transformer (LLM) For Question Asking

"***FROM SCRATCH***"

### Notebook Structure
---
**Disclaimer** :
- #### Data 
  - Data source
  - Tokenization
  - Features and Target
  - Test data
- #### Model Design
  - Positional encoding
  - Multi-head attention
  - Transformer Decoder
  - Final Architecture
- #### Training script
- #### Simplistic Inference Script
- #### Issues and mistakes
    - Pre-training with a downstream task
    - Not masking Padding layers
    - Context window
    

In [1]:
#necessary imports
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plot
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import random

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

GPU is available


## DATA

### Data Source
> The data I used for this project is the Stanford Question Ansewring Dataset (SQuAD). SQuAD was prepared such that a question and a context would map to an Answer (Q+C --> A). I modified this the data so that a Context would map to question (C --> Q).

**Find my modified data here** = [link to dataset](https://drive.google.com/file/d/1sxDIQPxQUxQ9P12lLHyGydnDyNcNwzS-/view?usp=sharing)


In [3]:
data = pd.read_json('{fill with path to your data}').to_dict(orient='list')

### Tokenization

In [ ]:
#bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [5]:
#Example
data['conversation'][0]

[{'from': 'human',
  'value': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'},
 {'from': 'gpt', 'value': 'When did Beyonce start becoming popular?'}]

In [6]:
def tokenize_input(qa):
    #1. tokenizing with a max seq length of 300 and padding layers
    #2. Adding an <sos> and <eos> token to target values. In this case; [CLS] and [SEP]
    seq_length = 300
    q_tokens = tokenizer(qa[0]['value'],add_special_tokens=False)['input_ids']
    a_tokens = tokenizer(qa[1]['value'],padding=True)['input_ids']


    x_tokens = q_tokens + a_tokens[:-1]
    y_tokens = q_tokens[1:] + a_tokens

    x_pad = [0 for i in range(seq_length-len(x_tokens))]
    y_pad = [0 for i in range(seq_length-len(x_tokens))]
    final_x = x_tokens + x_pad
    final_y = y_tokens + y_pad

    return final_x, final_y

In [7]:
#tokenizing all data
tokens = []
targets = []
for i in random.sample(data['conversation'],len(data['conversation'])):
    try:
        x, y = tokenize_input(i)

        if len(x) == 300:
            tokens.append(x)
            targets.append(y)
    except:
        pass


X = torch.IntTensor(tokens)
Y = torch.LongTensor(targets)

Token indices sequence length is longer than the specified maximum sequence length for this model (718 > 512). Running this sequence through the model will result in indexing errors


In [8]:
X.shape, Y.shape

(torch.Size([124975, 300]), torch.Size([124975, 300]))

## Test Data

#### **Create your test data here**

# Model Design

## Important Notes
- **Embedding layer:** I used the emmbedding layer from the bert model.
- **Positional encoding:** Sinusoidal encoding from Attention is all you need
- **Attention:** Multihead (4 heads)
- **Linear projection:** Projected input to 224 before passing it through decoder
- **Number of decoders:** 8

![image](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Ff6133c18-bfaf-4578-8c5a-e5ac7809f65b_1632x784.png)

### Embedding layer

In [9]:
class embed(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.embedder = AutoModel.from_pretrained('bert-base-uncased')

    def forward(self,x_tokens):
        inputs = {'input_ids':x_tokens}
        with torch.no_grad():
            attention_mask = (inputs['input_ids'] != 0).int()
            outputs = self.embedder(**inputs,attention_mask=attention_mask)
            embeddings = outputs.last_hidden_state * attention_mask.unsqueeze(-1)
        return embeddings


### Positional Encoding

In [10]:
## sinusoidal positional encoding

class pos_enc(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self,x):
        batch_size, max_seq_length, dmodel = x.shape
        pe = torch.zeros_like(x) #position encoding matrix

        # Compute the positional encoding values
        for pos in range(max_seq_length):
            for i in range(0, dmodel):
                if i % 2 == 0:
                    pe[:, pos, i] = torch.math.sin(pos / (10000 ** (2 * i / dmodel)))
                else:
                    pe[:, pos, i] = torch.math.cos(pos / (10000 ** (2 * i / dmodel)))

        x = x + pe
        return x




### Self-attention mechanisim

In [11]:
class self_attention(torch.nn.Module):
        def __init__(self,no_of_heads: int ,shape: tuple, mask: bool=False, QKV: list=[]):
                '''
        Initializes a Self Attention module as described in the "Attention is all you need" paper.
        This module splits the input into multiple heads to allow the model to jointly attend to information
        from different representation subspaces at different positions. After attention is applied independently
        on each head, the module concatenates and linearly transforms the results.




        ## Parameters:
            * no_of_heads (int): Number of attention heads. To implement single head attention, set this parameter to 1. ,i.e, no_of_heads = 1

           * shape (tuple): A tuple (seq_length, dmodel) where `seq_length` is the length of the input sequence,
                           and `dmodel` is the dimensionality of the input feature space.

            * mask (bool, optional): If True, a mask will be applied to prevent attention to future positions. This is particularly useful in decoder layers to ensure that the predictions for a sequence position can depend only on the known outputs at previous positions. Defaults to False.

            * QKV (list, optional): A list containing pre-computed Query (Q), Key (K), and Value (V) matrices. If provided, these matrices will be used instead of computing `Q`, `K`, and `V` from the input tensor. This is useful for operations where `Q`, `K`, and `V` come from different sources, such as in cross-attention in the Transformer decoder. The list should contain three tensors of shape (batch_size, seq_length, dmodel), corresponding to Q, K, and V, respectively.
        The forward pass computes the multi-head attention for input `x` and returns the transformed output.
                '''
                super().__init__()
                self.h = no_of_heads
                self.seq_length,self.dmodel = shape
                self.dk = self.dmodel//self.h
                self.softmax = torch.nn.Softmax(dim=-1)
                self.mQW = torch.nn.ModuleList([torch.nn.Linear(self.dmodel,self.dk) for i in range(self.h)])
                self.mKW = torch.nn.ModuleList([torch.nn.Linear(self.dmodel,self.dk) for i in range(self.h)])
                self.mVW = torch.nn.ModuleList([torch.nn.Linear(self.dmodel,self.dk) for i in range(self.h)])
                self.output_linear = torch.nn.Linear(self.dmodel,self.dmodel)
                self.mask = mask
                self.QKV = QKV

        def __add_mask(self,atten_values):
              #masking attention values
              mask_value = -1e9
              mask = torch.triu(torch.ones(atten_values.shape) * mask_value, diagonal=1)
              masked = atten_values + mask.to(device)
              return masked

        def forward(self, x):
            heads = []
            for i in range(self.h):
                # Apply linear projections in batch from dmodel => h x d_k
                if self.QKV:
                      q = self.mQW[i](self.QKV[0])
                      k = self.mKW[i](self.QKV[1])
                      v = self.mVW[i](self.QKV[2])
                else:
                        q = self.mQW[i](x)
                        k = self.mKW[i](x)
                        v = self.mVW[i](x)


                # Calculate attention using the projected vectors q, k, and v
                self.scores = torch.matmul(q, k.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.dk, dtype=torch.float32))
                if self.mask:
                      self.scores = self.__add_mask(self.scores)


                attn = self.softmax(self.scores)
                head_i = torch.matmul(attn, v)

                heads.append(head_i)

            # Concatenate all the heads together
            multi_head = torch.cat(heads, dim=-1)
            # Final linear layer
            output = self.output_linear(multi_head)

            return output + x  # Residual connection

### Decoder

In [12]:

class decoder_layer(torch.nn.Module):
    def __init__(self,shape: tuple,no_of_heads:int = 1):
        '''
        Implementation of Transformer Dencoder
        Parameters:
            shape (tuple): The shape (H, W) of the input tensor
            no_of_heads (int): number of heads in the attention mechanism. set this to 1 for single head attntion. default = 1
        Returns:
            Tensor: The output of the encoder layer after applying attention, feedforward network, and normalization.
        '''
        super().__init__()

        self.max_seq_length,self.dmodel = shape
        def ff_weights():
            layer1 =  torch.nn.Linear(self.dmodel,600)
            layer2 = torch.nn.Linear(600,600)
            layer3 = torch.nn.Linear(600,self.dmodel)
            return layer1,layer2,layer3

        self.no_of_heads = no_of_heads

        self.multi_head =  self_attention(no_of_heads=no_of_heads, mask=True,
                                                    shape=(self.max_seq_length,self.dmodel))

        self.layer1,self.layer2,self.layer3 = ff_weights()
        self.softmax = torch.nn.Softmax(dim=-1)
        self.layerNorm = torch.nn.LayerNorm(shape)
        self.relu1 = torch.nn.ReLU()
        self.relu2 = torch.nn.ReLU()

    def feed_forward(self,x):
        f = self.layer1(x)
        f = self.relu1(f)
        f = self.layer2(f)
        f = self.relu2(f)
        f = self.layer3(f)

        return self.layerNorm(f  + x) #residual connection

    def forward(self,x):
        x = self.multi_head(x)
        x = self.layerNorm(x)
        x = self.feed_forward(x)
        x = self.layerNorm(x)

        return x

### Full Model Architecture

In [13]:
class architecture(torch.nn.Module):
    def __init__(self,n_classes,shape) -> None:
        super().__init__()
        self.max_seq_length,self.dmodel = shape
        self.projected_dmodel = 224
        self.embedding_layer = embed()
        self.proj_to_224 = torch.nn.Linear(self.dmodel, self.projected_dmodel)
        self.positional = pos_enc()
        self.decoder1 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder2 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder3 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder4 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder5 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder6 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder7 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        self.decoder8 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel), no_of_heads=8)
        # self.decoder5 = decoder_layer(shape=(self.max_seq_length,self.projected_dmodel))
        self.final_MLP = torch.nn.Linear(self.projected_dmodel,n_classes)
        self.softmax = torch.nn.Softmax(dim=2)

    def forward(self,x,temperature=1.0):
        x = self.embedding_layer(x)
        x = self.proj_to_224(x)
        x = self.positional(x)
        x = self.decoder1(x)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)
        x = self.decoder5(x)
        x = self.decoder6(x)
        x = self.decoder7(x)
        x = self.decoder8(x)
        x = self.final_MLP(x)
        logits = x / temperature
        x = self.softmax(logits)


        return x

# Training Script

In [66]:
!pip install torchmetrics

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from torchmetrics import Accuracy
from tqdm import tqdm

In [15]:
dataset = torch.utils.data.TensorDataset(X,Y)
loader = torch.utils.data.DataLoader(dataset,batch_size=20,num_workers=0,shuffle=False)

In [16]:
vocab_size = tokenizer.vocab_size
model = architecture(n_classes = vocab_size, shape = (300,768))
model = model.to(device)
model.load_state_dict(torch.load("{fill with path to model weights if any}"))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

<All keys matched successfully>

In [76]:
metric =  Accuracy(num_classes=vocab_size,task='multiclass').to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
criterion = torch.nn.CrossEntropyLoss(ignore_index=0,label_smoothing=0.01)

In [ ]:
from tqdm import tqdm

print('Training Started')
NUM_EPOCHS = 1

for epoch in range(NUM_EPOCHS):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    epoch_accuracy = 0.0
    num_batches = len(loader)

    # Initialize tqdm progress bar
    with tqdm(total=num_batches, desc=f"Epoch {epoch + 1}", leave=True) as pbar:
        for i, (x_batch, y_batch) in enumerate(loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(x_batch)
            # Flatten the outputs and y_batch tensors one dimension lower
            outputs = outputs.view(-1, outputs.shape[-1])
            y_batch = y_batch.view(-1)
            
            # Loss calculation
            loss = criterion(outputs, y_batch).to(device)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Metrics
            argmax_pred = outputs.argmax(axis=1)
            metric.update(argmax_pred, y_batch)

            # Print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # update every 10 mini-batches
                accuracy = metric.compute().item()
                epoch_accuracy += accuracy
                pbar.set_postfix({'Loss': running_loss / (i + 1), 'Accuracy': accuracy})
            
            # Update the progress bar
            pbar.update(1)

            # Save model weights periodically
            if i % 10 == 9:
                torch.save(model.state_dict(), '/kaggle/working/model_weights.pth')

    # Compute and print average loss and accuracy for the epoch
    avg_loss = running_loss / num_batches
    avg_accuracy = epoch_accuracy / (num_batches // 10)  # since we're summing accuracy every 10 batches
    print(f'Epoch {epoch + 1} - Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}')

print('Training Completed')


Training Started


Epoch 1:  66%|██████▋   | 2770/4166 [1:58:07<1:00:08,  2.59s/it, Loss=9.89, Accuracy=0.234]

In [ ]:
#Link to download model
from IPython.display import FileLink
FileLink(r'model_weights.pth')

# Simplistic Inference Script

In [17]:
text = '''Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'''

In [6]:
def model_pred(tokens,temp):
    model.eval()
    with torch.no_grad():
        pred = model(tokens,temp)
        pred = pred.view(-1, pred.shape[-1]).argmax(axis=1)
    return pred

def tokenize_text(text):
    seq_length = 300
    q_tokens = tokenizer(text,add_special_tokens=False)['input_ids']
    pad = [0 for i in range(seq_length-len(q_tokens))]
    final_tokens = [q_tokens + pad]
    last_index = len(q_tokens)-1
    
    return torch.tensor(final_tokens),last_index

def inference(text, starter='', temperature=1.0):
    curr = 0
    pred_list = []
    t, last_token = tokenize_text(text + '[CLS]' + starter)
    t = t.to(device)
    
    while curr != 102:
        print('\n',"Generating...")
        all_pred = model_pred(t, temperature)
        pred = all_pred[last_token].item()
        pred_list.append(pred)
        t[0][last_token + 1] = pred
        last_token += 1
        curr = pred
        
        if curr > 10:
            break
    print("Question from the model: ".upper(), starter + ' ' + tokenizer.decode(pred_list),'\n')

    return starter + ' ' + tokenizer.decode(pred_list)

inference(text, '')



QUESTION FROM THE MODEL:  what is the name of the singer? [SEP]


In [30]:
tokenizer.decode(X[1:2][0])

'in some countries, the personal copying exemption explicitly requires that the content being copied was obtained legitimately – i. e., from authorized sources, not file - sharing networks. other countries, such as the netherlands, make no such distinction ; the exemption there had been assumed, even by the government, to apply to any such copying, even from file - sharing networks. however, in april 2014, the court of justice of the european union ruled that " national legislation which makes no distinction between private copies made from lawful sources and those made from counterfeited or pirated sources cannot be tolerated. " thus, in the netherlands, for example, downloading from file - sharing networks is no longer legal. [CLS] what does the personal copying exemption explicitly need? [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

In [29]:
pred = model(X[1:2].to(device))
tokens = pred.view(-1, pred.shape[-1]).argmax(axis=1)
tokenizer.decode(tokens)

'the areas, the death use act is is that the data that used was not forly, that.,., from, witnesses, not light - in system. other states, including as the united, have a other relationship, the act in had been been, as as the government, to have to a other use, including from public - in system. in, in september 2011, the school of state of the united union was that " the law which is more fact from death witnesses from from non forces and the that from non -ed and nond species is be be. " for, in the united, for example, theing from non - in system is not more a. [CLS] what has the national use act have have? [SEP] the - many - - - - - people. - state [CLS] many the [CLS] [CLS].. to the in -.. the -er the many. [SEP] the [CLS] [SEP] what the the number to. power in s\'power. -azi [SEP] - - -. - -\'[SEP] the power the in [SEP] the government [SEP] power - [CLS] the [SEP] - population church %, in - the - power power - of - aser power power of power - as power the thely power in in iner

In [ ]:
model(X[1:2].to(device),temperature = 0.2).view(-1, outputs.shape[-1]).argmax(axis=1)

In [ ]:
torch.argmax(model(X[:3].to(device)),dim=1)

# Issues

- **Not Enough data:** I trained on only 130K samples which is too small
- **Pre-trainig on a downstream task:** Pretraining is supposed to be self supervised
- **Not masking the padding tokens**
